In [1]:
import sys
sys.path.append('./source_code')

In [2]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, learning_curve, cross_val_score, \
    KFold, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, \
    roc_auc_score, roc_curve, confusion_matrix
import xgboost as xgb
import classification_helpers as ch

%config InlineBackend.figure_format = 'png'
%matplotlib inline
sns.set(color_codes=True)
plt.style.use('seaborn-colorblind')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 2)
pd.set_option('display.max_info_columns', 200)

In [3]:
with open('pickles/combined_sample_weather_df5.pickle', 'rb') as read_file:
    sample_df = pickle.load(read_file)

In [4]:
with open('pickles/cv_records_df.pickle', 'rb') as read_file:
    cv_records_df = pickle.load(read_file)

In [5]:
base_numerical_features = ['airline_avg_arrival_delay',
                           'origin_airport_avg_flights_per_day',
                           'origin_airport_avg_departure_delay',
                           'destination_airport_avg_flights_per_day',
                           'destination_airport_avg_arrival_delay',
                           'month_avg_delay',
                           'day_of_week_avg_delay']

weather_numerical_features = ['Precipitation_origin',
                              'Snow Depth_origin',
                              'Visibility_origin',
                              'Wind Speed_origin',                      
                              'Precipitation_destination',
                              'Snow Depth_destination',
                              'Visibility_destination',
                              'Wind Speed_destination']
                              

weather_categorical_features = ['is_Clear_origin',
                                'is_Partially cloudy_origin',
                                'is_Rain_origin',
                                'is_Snow_origin',
                                'is_Overcast_origin',
                                'is_Clear_destination',
                                'is_Partially cloudy_destination',
                                'is_Rain_destination',
                                'is_Snow_destination',
                                'is_Overcast_destination']

base_categorical_features = ['American Airlines Inc.', 'American Eagle Airlines Inc.',
                             'Atlantic Southeast Airlines', 'Delta Air Lines Inc.',
                             'Frontier Airlines Inc.', 'Hawaiian Airlines Inc.', 'JetBlue Airways',
                             'Skywest Airlines Inc.', 'Southwest Airlines Co.', 'Spirit Air Lines',
                             'US Airways Inc.', 'United Air Lines Inc.', 'Virgin America',
                             'is_thanksgiving_period', 'is_xmas_period',
                             'is_night_departure', 'is_night_arrival']

target = ['is_not_on_time']

all_features = (base_numerical_features 
                + base_categorical_features 
                + weather_numerical_features)

In [6]:
X, y = ch.feature_target_selection(all_features, target, sample_df)

X_train_val, X_test, y_train_val, y_test = ch.initial_split(X, y.values.ravel())
X_train, X_val, y_train, y_val = ch.second_split(X_train_val, y_train_val)

scaler = StandardScaler()
X_train_val_scaled = scaler.fit_transform(X_train_val)

lm = LogisticRegression(penalty='none', max_iter=10000)
rf = RandomForestClassifier()

In [7]:
cv_records_df.tail()

,model,iteration_desc,feature_engineering,hyperparameter_tuning,mean_train_f1,mean_val_f1,mean_train_precision,mean_val_precision,mean_train_recall,mean_val_recall,mean_train_accuracy,mean_val_accuracy,mean_train_AUC,mean_val_AUC
21,LogisticRegression,,added weather_numerical_features,"max_iter=10000, penalty='none'",0.11,0.10,0.60,0.59,0.06,0.06,0.80,0.80,0.67,0.67
22,LogisticRegression,,took out temperature/cloud cover features beca...,"max_iter=10000, penalty='none'",0.11,0.11,0.60,0.60,0.06,0.06,0.80,0.80,0.67,0.67
23,LogisticRegression,,added weather_categorical_features,"max_iter=10000, penalty='none'",0.11,0.11,0.61,0.61,0.06,0.06,0.80,0.80,0.67,0.67
24,LogisticRegression,scaled features,,"max_iter=10000, penalty='none'",0.11,0.11,0.60,0.60,0.06,0.06,0.80,0.80,0.67,0.67
25,RandomForestClassifier,Used default RandomForestClassifier hyperparam...,,,0.97,0.20,0.99,0.57,0.94,0.12,0.99,0.81,0.99,0.68


1. Tune max_depth
2. Tune subsample
3. Tune min_child_weight
4. Tune colsample_bytree
5. Set learning_rate to .05 and use early stopping to get n_estimators

In [11]:
gbm = xgb.XGBClassifier(
    n_estimators=500,  # arbitrary large number
    max_depth=6,
    objective='binary:logistic',
    learning_rate=.1,
    subsample=.7,
    min_child_weight=1,
    colsample_bytree=1
)

eval_set = [(X_train, y_train), (X_val, y_val)]

gbm.fit(
    X_train, y_train, 
    eval_set=eval_set,
    eval_metric='auc',
    early_stopping_rounds=20,
    verbose=True
)

[0]	validation_0-auc:0.66351	validation_1-auc:0.64686
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 20 rounds.
[1]	validation_0-auc:0.67172	validation_1-auc:0.65390
[2]	validation_0-auc:0.67705	validation_1-auc:0.65875
[3]	validation_0-auc:0.68059	validation_1-auc:0.66291
[4]	validation_0-auc:0.68244	validation_1-auc:0.66469
[5]	validation_0-auc:0.68482	validation_1-auc:0.66568
[6]	validation_0-auc:0.68704	validation_1-auc:0.66768
[7]	validation_0-auc:0.68831	validation_1-auc:0.66801
[8]	validation_0-auc:0.69048	validation_1-auc:0.66880
[9]	validation_0-auc:0.69202	validation_1-auc:0.66970
[10]	validation_0-auc:0.69409	validation_1-auc:0.67133
[11]	validation_0-auc:0.69608	validation_1-auc:0.67208
[12]	validation_0-auc:0.69797	validation_1-auc:0.67305
[13]	validation_0-auc:0.69920	validation_1-auc:0.67431
[14]	validation_0-auc:0.70072	validation_1-auc:0.67572
[15]	validation_0-auc:0.7026

[146]	validation_0-auc:0.78299	validation_1-auc:0.69881
[147]	validation_0-auc:0.78326	validation_1-auc:0.69875
[148]	validation_0-auc:0.78363	validation_1-auc:0.69869
[149]	validation_0-auc:0.78404	validation_1-auc:0.69865
[150]	validation_0-auc:0.78452	validation_1-auc:0.69893
[151]	validation_0-auc:0.78470	validation_1-auc:0.69888
[152]	validation_0-auc:0.78510	validation_1-auc:0.69894
[153]	validation_0-auc:0.78531	validation_1-auc:0.69886
[154]	validation_0-auc:0.78574	validation_1-auc:0.69886
[155]	validation_0-auc:0.78651	validation_1-auc:0.69922
[156]	validation_0-auc:0.78696	validation_1-auc:0.69906
[157]	validation_0-auc:0.78718	validation_1-auc:0.69919
[158]	validation_0-auc:0.78751	validation_1-auc:0.69910
[159]	validation_0-auc:0.78797	validation_1-auc:0.69894
[160]	validation_0-auc:0.78846	validation_1-auc:0.69889
[161]	validation_0-auc:0.78874	validation_1-auc:0.69890
[162]	validation_0-auc:0.78892	validation_1-auc:0.69890
[163]	validation_0-auc:0.78934	validation_1-auc:

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.7,
              tree_method=None, validate_parameters=False, verbosity=None)

In [17]:
cv_records_df = ch.cv(gbm, X_train_val_scaled, y_train_val, cv_records_df)

Model name: XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.7,
              tree_method=None, validate_parameters=False, verbosity=None)
Hyperparameters: 

Mean train F1:             0.478
Mean val F1:               0.246
Mean train precision:      0.937
Mean val precision:        0.569
Mean train recall:         0.321
Mean val recall:           0.157
Mean train accuracy:       0.859
Mean val accuracy:         0.807
Mean train AUC:            0.873
Mean val AUC:              0.690
iteration_desc: testing xgboost for first time
feature_engineeri

In [22]:
cv_records_df.reset_index(inplace=True)

In [9]:
sv_records_df = pd.DataFrame()

In [18]:
gbm.predict(X_val)

array([0.09191132, 0.5749895 , 0.12496357, ..., 0.22924785, 0.17216212,
       0.17589705], dtype=float32)

In [9]:
f1_score(y_val, gbm.predict(X_val))

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [10]:
_, sv_records_df = ch.split_and_simple_validate(gbm, X_train_val, y_train_val, 
                                                sv_records_df, threshold=0.5,
                                                xgboost=True)

Train F1:                                 0.37
Val F1:                                   0.23
Train precision:                          0.89
Val precision:                            0.62
Train recall:                             0.23
Val recall:                               0.14
Train accuracy:                           0.84
Val accuracy:                             0.81
Train AUC:                                0.61
Val AUC:                                  0.56
iteration_desc: testing xgboost
feature_engineering: 
